
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EffiSciencesResearch/hackathon42/blob/main/starting_pack_hackathon42.ipynb)


In [8]:
# This can take a moment
# Please install git large file system before cloning
!git lfs install
!git clone https://github.com/EffiSciencesResearch/hackathon42.git

Updated Git hooks.
Git LFS initialized.
Cloning into 'hackathon42'...
remote: Repository not found.
fatal: repository 'https://github.com/EffiSciencesResearch/hackathon42.git/' not found
zsh:cd:1: no such file or directory: hackathon42


In [ ]:
%cd /content/hackathon42

import numpy as np
from tqdm.notebook import tqdm
import os
from os.path import join
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

datasets_names = os.listdir(path='datasets')

# You can choose the dataset here by modifying the brackets
dataset_name = join('datasets', datasets_names[1])
print(dataset_name)

X_labeled = np.load(join(dataset_name, "X_labeled.npy"))
y_labeled = np.load(join(dataset_name, "y_labeled.npy"))
X_unlabeled = np.load(join(dataset_name, "X_unlabeled.npy"))
X_val = np.load(join(dataset_name, "X_val.npy"))


X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled)


X_train = torch.tensor(X_train, device=device).float()
X_test = torch.tensor(X_test, device=device).float()
y_train = torch.tensor(y_train, device=device)
y_test = torch.tensor(y_test, device=device)

In [ ]:
class LeNet(nn.Module):
    def __init__(self, n_classes=10):
        super().__init__()       
        
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32*4*11, 100)
        self.fc2 = nn.Linear(100, 10)
        self.flat = nn.Flatten()
        
    def forward(self, x):
        
        # Max pooling over a (2, 2) window
        out = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)
        out = self.flat(out)
        # out = F.dropout(out, 0.5)
        out = F.relu(self.fc1(out))
        # out = F.dropout(out, 0.25)
        pred = self.fc2(out)
        
        return pred

net = LeNet().float()

In [ ]:
tbatch = 100
nb_epochs = 20

nbbatchs = int(X_train.shape[0]/tbatch)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

net.train()

def evaluate(X_test):
    net.eval()
    with torch.no_grad():
        y_pred = net(X_test)
    correct_test = (torch.argmax(y_pred, axis=1) == y_test).sum().item()
    net.train()
    return correct_test/len(X_test)

for epoch in tqdm(range(nb_epochs)):  # loop over the dataset multiple times
    correct = 0
    for i in (range(nbbatchs)):
        
        optimizer.zero_grad()
        
        X_batch = X_train[i*tbatch:(i+1)*tbatch]
        y_batch = y_train[i*tbatch:(i+1)*tbatch]

        # Compute forward / backward
        y_pred = net(X_batch)
        correct += (torch.argmax(y_pred, axis=1) == y_batch).sum().item()

        loss = criterion(y_pred, y_batch)
        

        loss.backward()
        optimizer.step()
        # break
    print("acc train:", correct/X_train.shape[0], "acc test:", evaluate(X_test))




  0%|          | 0/20 [00:00<?, ?it/s]

acc train: 0.7431578947368421 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
acc train: 0.9473684210526315 acc test: 1.0
